In [39]:
import os
from dotenv import load_dotenv
import psycopg2
import numpy as np
from langchain_google_genai import GoogleGenerativeAIEmbeddings

load_dotenv(override= True)

True

In [14]:
texts = [
    "Type: Desktop, OS: Ubuntu, GPU: NVIDIA, CPU: AMD, RAM: 64GB, SSD: 2TB",
    "Type: Desktop, OS: Linux Mint, GPU: NVIDIA, CPU: AMD, RAM: 64GB, SSD: 2TB",
    "Type: Desktop, OS: Manjaro, GPU: NVIDIA, CPU: AMD, RAM: 64GB, SSD: 2TB",
    "Type: Desktop, OS: Windows, GPU: NVIDIA, CPU: AMD, RAM: 64GB, SSD: 2TB",
    "Type: Desktop, OS: Windows, GPU: NVIDIA, CPU: Intel, RAM: 32GB, SSD: 1TB",
    "Type: Desktop, OS: Fedora, GPU: AMD, CPU: AMD, RAM: 16GB, SSD: 1TB",
    "Type: Desktop, OS: Windows, GPU: NVIDIA, CPU: AMD, RAM: 16GB, SSD: 2TB",
    "Type: Desktop, OS: Windows, GPU: AMD, CPU: AMD, RAM: 16GB, SSD: 2TB",
    "Type: Desktop, OS: Ubuntu, GPU: NVIDIA, CPU: AMD, RAM: 32GB, SSD: 1TB",
    "Type: Laptop, OS: Windows, GPU: NVIDIA, CPU: Intel, RAM: 16GB, SSD: 1TB",
    "Type: Laptop, OS: Ubuntu, GPU: AMD, CPU: AMD, RAM: 16GB, SSD: 500GB",
    "Type: Laptop, OS: Mac OS, GPU: NVIDIA, CPU: AMD, RAM: 16GB, SSD: 1TB",
]


In [40]:
gemini_api_key= os.getenv("GOOGLE_API_KEY")

In [41]:
embeddings = GoogleGenerativeAIEmbeddings(model= "models/embedding-001")

In [17]:
embeddings_list = []
for text in texts:
    embeddings_list.append(embeddings.embed_query(text))

In [24]:
print(len(embeddings_list[0]))

768


In [32]:
connection = psycopg2.connect(dbname="vectordbtutorial", user="postgres", password="postgres", host="localhost", port=5433)
cursor = connection.cursor()

for i in range(len(embeddings_list)):
    embedding = embeddings_list[i]
    content = texts[i]
    cursor.execute("INSERT INTO items (content, embedding) VALUES (%s, %s);", (content, embedding))
connection.commit()

print(cursor.execute("SELECT * FROM items;"))

cursor.close()
connection.close()

None


In [ ]:

new_text = "Type: Desktop, OS: Arch Linux, GPU: NVIDIA, CPU: AMD, RAM: 64GB, SSD: 2TB"
new_embedding = embeddings.embed_query(new_text)

[0.024523787,
 -0.021914156,
 -0.03416501,
 -0.01772358,
 0.047045875,
 0.00044630852,
 0.019791886,
 -0.02220959,
 0.022866359,
 0.04613413,
 0.02942512,
 0.012496917,
 -0.03946748,
 -0.00019881428,
 0.022581799,
 -0.036057416,
 -0.020235727,
 0.014056601,
 0.018632479,
 0.024524126,
 0.00899194,
 0.040017787,
 0.0402326,
 -0.016409673,
 -0.03522131,
 -0.03426008,
 0.025353195,
 -0.02971229,
 -0.055265512,
 0.044269647,
 -0.080744684,
 0.0146287065,
 -0.0811804,
 0.010531601,
 0.0001662045,
 -0.022323847,
 0.0038092548,
 -0.015102232,
 -0.009479383,
 0.031504545,
 0.022009922,
 -0.037672073,
 -0.03929481,
 -0.018356852,
 0.04550699,
 0.0006742126,
 -0.009659821,
 -0.0077497116,
 0.019213056,
 -0.036690213,
 0.041172244,
 -0.048430346,
 -0.0016223075,
 0.0032073855,
 -0.020183966,
 -0.031535972,
 -0.006607235,
 0.009265048,
 0.006164374,
 -0.024678802,
 0.048009664,
 0.010705766,
 -0.019297805,
 0.091571756,
 0.025176287,
 -0.04700554,
 -0.032998662,
 -0.0028031594,
 0.08824253,
 -0.05

In [51]:
connection = psycopg2.connect(dbname="vectordbtutorial", user="postgres", password="postgres", host="localhost", port=5433)
cursor = connection.cursor()

cursor.execute("""
    SELECT id, content
    FROM items
    ORDER BY embedding <-> %s::vector
    LIMIT 5;
""", (new_embedding,))

In [52]:
result = cursor.fetchall()
for res in result:
    print(res)

(13, 'Type: Desktop, OS: Ubuntu, GPU: NVIDIA, CPU: AMD, RAM: 64GB, SSD: 2TB')
(1, 'Type: Desktop, OS: Ubuntu, GPU: NVIDIA, CPU: AMD, RAM: 64GB, SSD: 2TB')
(21, 'Type: Desktop, OS: Ubuntu, GPU: NVIDIA, CPU: AMD, RAM: 32GB, SSD: 1TB')
(9, 'Type: Desktop, OS: Ubuntu, GPU: NVIDIA, CPU: AMD, RAM: 32GB, SSD: 1TB')
(2, 'Type: Desktop, OS: Linux Mint, GPU: NVIDIA, CPU: AMD, RAM: 64GB, SSD: 2TB')


In [54]:
connection.close()
cursor.close()